# Forecasting examples


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from copy import copy

from foressment_ai.forecasting.forecaster_ai.loader import DataLoaderAndPreprocessorDefault
from foressment_ai.forecasting.forecaster_ai.forecaster import *

Using TensorFlow backend


### Download and prepare the dataset.

In [3]:
dataset_names = ['etd_m1', 'etd_m2', 'etd_h1', 'etd_h2']
datasets = {}

In [4]:
for dataset_name in dataset_names:
    dataset = DataLoaderAndPreprocessorDefault()
    dataset.load_data(dataset_name)
    dataset.set_train_size(0.8)

    datasets[dataset_name] = {}
    datasets[dataset_name]['train'] = dataset.get_train_data()
    datasets[dataset_name]['test']  = dataset.get_test_data()

    print(f"{dataset_name}: train = {datasets[dataset_name]['train'].data.shape}, test = {datasets[dataset_name]['test'].data.shape}")

    datasets[dataset_name]['train'].scale()
    datasets[dataset_name]['test'].scale(scaler=datasets[dataset_name]['train'].scaler)

etd_m1: train = (55744, 7), test = (13936, 7)
etd_m2: train = (55744, 7), test = (13936, 7)
etd_h1: train = (13936, 7), test = (3484, 7)
etd_h2: train = (13936, 7), test = (3484, 7)


Create time series generators.

In [5]:
look_back_length = 90
horizon = 1
n_features = 7

model_params = AIForecasterParameters(n_features=n_features,
                                      look_back_length=look_back_length,
                                      horizon=horizon)

tsgenerators = {}
for dataset_name, values in datasets.items():
    tsgenerators[dataset_name] = {}
    tsgenerators[dataset_name]['train'] = TSGenerator(values['train'].data, model_params)
    tsgenerators[dataset_name]['test']  = TSGenerator(values['test'].data, model_params)
    print('Train timeseries:', tsgenerators[dataset_name]['train'].data.shape, '->', tsgenerators[dataset_name]['train'].targets.shape)
    print('Test timeseries:', tsgenerators[dataset_name]['test'].data.shape, '->', tsgenerators[dataset_name]['test'].targets.shape)

Train timeseries: (55654, 90, 7) -> (55654, 1, 7)
Test timeseries: (13846, 90, 7) -> (13846, 1, 7)
Train timeseries: (55654, 90, 7) -> (55654, 1, 7)
Test timeseries: (13846, 90, 7) -> (13846, 1, 7)
Train timeseries: (13846, 90, 7) -> (13846, 1, 7)
Test timeseries: (3394, 90, 7) -> (3394, 1, 7)
Train timeseries: (13846, 90, 7) -> (13846, 1, 7)
Test timeseries: (3394, 90, 7) -> (3394, 1, 7)


### Keras Tuner

In [ ]:
tuner = AIForecasterTuner(model_params)
tuned_hps = {
             'block_type': ['LSTM', 'GRU'],
             'n_rec_layers': [1, 2, 3],
             'units': [[512, 256, 180], [128, 80, 64], [32, 16, 8]],
             'dropout': [0.0, 0.01],
             'hidden_activation': ['tanh', 'relu'],
             'output_activation': ['linear', 'sigmoid']
            }
tuner.set_tuned_hps(block_type=tuned_hps['block_type'],
                    units=tuned_hps['units'],
                    n_rec_layers=tuned_hps['n_rec_layers'],
                    dropout=tuned_hps['dropout'],
                    hidden_activation=tuned_hps['hidden_activation'],
                    output_activation=tuned_hps['output_activation'])
print(tuner.hp_choices)

In [ ]:
for dataset_name, values in tsgenerators.items():
    ltuner = copy(tuner)
    bests = ltuner.find_best_models(values['train'].get_data(), values['train'].get_targets(),
                           tuner_type='BayesianOptimization',
                           max_trials=10,
                           epochs=5,
                           n_models=5,
                           batch_size=256)
    for i, model in enumerate(bests):
        filename = f'ai_models/configs/{dataset_name}_{model.default_filename}_{i}.json'
        model.save_model_config(filename)
        print(f'Model {i} configuration save to', filename)

In [ ]:
import os

In [ ]:
best_models = []
for file in os.listdir('ai_models/configs/etd')[:1]:
    print(file)
    model = AIForecaster(from_file_config='ai_models/configs/etd/' + file)
    best_models.append(model)

In [ ]:
batch_size = 256
for dataset_name, values in tsgenerators.items():
    for i, model in enumerate(best_models):
        print(dataset_name, model.default_filename)
        model.train(X=values['train'].get_data(), 
                    y=values['train'].get_targets(),
        batch_size=batch_size, n_epochs=5, validation_split=0.1)
    
        filename = 'ai_models/ready/' + dataset_name + '/' + model.default_filename + f'_{i}.keras'
        model.save_model(filename)
        print('Model save to', filename)
        print('')

In [ ]:
nm = NaiveForecaster(model_params)
forecasting_data_lengths = [336, 720]

for forecasting_data_length in forecasting_data_lengths:
    for dataset_name, values in tsgenerators.items():
        if (dataset_name in ['etd_m1', 'etd_m2', 'etd_h1']) and (forecasting_data_length==336):
            continue
        test_values = copy(values['test'])
        test_values.change_horizon(forecasting_data_length)
        
        test_values.data = test_values.data[::forecasting_data_length]
        test_values.targets = test_values.targets[::forecasting_data_length]
        
        print(dataset_name, test_values.data.shape, test_values.targets.shape)
        current_batch = test_values.get_data()
        estimator = ForecastEstimator()
        estimator.set_true_values(test_values.get_targets())
        
        for file in os.listdir('ai_models/ready/' + dataset_name):
            print(file)
            model = AIForecaster(from_file='ai_models/ready/' + dataset_name + '/' + file)
            pred = model.forecasting(current_batch,
               forecasting_data_length=forecasting_data_length)
            estimator.set_pred_values(pred, model_name=model.default_filename)
        
        naive_pred = nm.forecasting(current_batch, forecasting_data_length=forecasting_data_length)
        estimator.set_pred_values(naive_pred, model_name='naive')
        
        estimator.estimate()
        estimator.save_quality(f'{dataset_name}_quality_{forecasting_data_length}')

etd_h2 (10, 90, 7) (10, 336, 7)
gru_180_128_32_d0_3.keras

Forecasting: 100%|██████████| 10/10 [02:29<00:00, 14.96s/it]
gru_256_128_d0_6.keras

Forecasting: 100%|██████████| 10/10 [02:27<00:00, 14.73s/it]
gru_256_d0_7.keras

Forecasting: 100%|██████████| 10/10 [02:18<00:00, 13.86s/it]
gru_512_80_d0_4.keras

Forecasting: 100%|██████████| 10/10 [02:51<00:00, 17.15s/it]
gru_512_d0_0.keras

Forecasting: 100%|██████████| 10/10 [02:41<00:00, 16.11s/it]
lstm_256_128_32_d0_5.keras

Forecasting: 100%|██████████| 10/10 [02:42<00:00, 16.30s/it]
lstm_512_d0_1.keras

Forecasting: 100%|██████████| 10/10 [02:56<00:00, 17.66s/it]

Forecasting: 100%|██████████| 10/10 [00:00<00:00, 1861.49it/s]
etd_m1 (19, 90, 7) (19, 720, 7)
gru_180_128_32_d0_3.keras


C:\Users\dmitr\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\metrics\_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\dmitr\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\metrics\_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\dmitr\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\metrics\_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\dmitr\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\metrics\_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\dmitr\PycharmProjects\pythonProject\venv\lib\site-p


Forecasting: 100%|██████████| 19/19 [10:36<00:00, 33.52s/it]
gru_256_128_d0_6.keras

Forecasting: 100%|██████████| 19/19 [11:21<00:00, 35.87s/it]
gru_256_d0_7.keras

Forecasting: 100%|██████████| 19/19 [12:42<00:00, 40.13s/it]
gru_512_80_d0_4.keras

Forecasting: 100%|██████████| 19/19 [16:48<00:00, 53.08s/it]
gru_512_d0_0.keras

Forecasting: 100%|██████████| 19/19 [15:41<00:00, 49.54s/it]
lstm_256_128_32_d0_5.keras

Forecasting: 100%|██████████| 19/19 [16:41<00:00, 52.71s/it]
lstm_512_d0_1.keras

Forecasting: 100%|██████████| 19/19 [18:59<00:00, 59.99s/it]

Forecasting: 100%|██████████| 19/19 [00:00<00:00, 337.07it/s]
etd_m2 (19, 90, 7) (19, 720, 7)
gru_180_128_32_d0_3.keras

Forecasting: 100%|██████████| 19/19 [18:46<00:00, 59.29s/it]
gru_256_128_d0_6.keras

Forecasting: 100%|██████████| 19/19 [20:27<00:00, 64.60s/it]
gru_256_d0_7.keras

Forecasting:  47%|████▋     | 9/19 [09:55<11:05, 66.58s/it]